# Book Success Machine Learning

Kathryn Hamilton and Frank Shannon

w207 Spring 2018

The team would like to assess the relationship between the synopsis of a novel and its success by constructing a classifier.

The synopsis of a novel, a couple paragraphs of text found on the back or inside cover of a novel, provide a brief explanation of the book's genre and contents, and often some critical acclaim of the the author if applicable. The team would like to see if this information can be used to reliably predict whether or not the novel will be successful.

We begin by importing the necessary libraries and setting up our document

In [15]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd
import gzip

The data we will be using for this project come from two sources.

The first is an online repository of .json files compiled by Julian McAuley, Assistant Professor of Computer Science and Engineering at University of California, San Diego. These files, which can be found at http://jmcauley.ucsd.edu/data/amazon/ will provides us with customer review information and product metadata.

The second will be an API the team uses to scrape book synopsis data from http://www.amazon.com/ using the list of product ID numbers included in the dataset of reviews.

In [16]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0 
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

metadata = getDF('meta_Books.json.gz')

# https://help.parsehub.com/hc/en-us/articles/219619888-Analyzing-JSON-With-Python

In [23]:
metadata

,asin,salesRank,imUrl,categories,title,description,related,price,brand
0,0001048791,{u'Books': 6334800},http://ecx.images-amazon.com/images/I/51MKP0T4...,[[Books]],"The Crucible: Performed by Stuart Pankin, Jero...",NaN,NaN,NaN,NaN
1,0001048775,{u'Books': 13243226},http://ecx.images-amazon.com/images/I/5166EBHD...,[[Books]],Measure for Measure: Complete &amp; Unabridged,William Shakespeare is widely regarded as the ...,NaN,NaN,NaN
2,0001048236,{u'Books': 8973864},http://ecx.images-amazon.com/images/I/51DH145C...,[[Books]],The Sherlock Holmes Audio Collection,"&#34;One thing is certain, Sherlockians, put a...","{u'also_viewed': [u'1442300191', u'9626349786'...",9.26,NaN
3,0000401048,{u'Books': 6448843},http://ecx.images-amazon.com/images/I/41bchvIf...,[[Books]],The rogue of publishers' row;: Confessions of ...,NaN,{u'also_viewed': [u'068240103X']},NaN,NaN
4,0001019880,{u'Books': 9589258},http://ecx.images-amazon.com/images/I/61LcHUdv...,[[Books]],Classic Soul Winner's New Testament Bible,NaN,"{u'also_viewed': [u'B003HMB5FC', u'0834004593'...",5.39,NaN
5,0001048813,NaN,http://ecx.images-amazon.com/images/I/41k5u0lr...,[[Books]],Archer Christmas 4 Tape Pack,NaN,NaN,NaN,NaN
6,0001148427,{u'Books': 5806769},http://ecx.images-amazon.com/images/I/41tN4KuO...,[[Books]],Sonatas - For Piano,NaN,NaN,NaN,NaN
7,0001057170,{u'Books': 9318563},http://ecx.images-amazon.com/images/I/51M65KR8...,[[Books]],Classic Connolly Boxed Set (Vol 1 &amp; 2),[Editor's Note: The following is a combined re...,NaN,NaN,NaN
8,0001047566,{u'Books': 3628249},http://ecx.images-amazon.com/images/I/51FWARNT...,[[Books]],Hand in Glove,NaN,NaN,NaN,NaN
9,0001053396,{u'Books': 12249714},http://ecx.images-amazon.com/images/I/51WTKK4V...,[[Books]],War Poems: An Anthology of Poetry from the 18t...,Writing poetry has always been a way to expres...,NaN,17.99,NaN


In [21]:
metadata.axes

[Int64Index([      0,       1,       2,       3,       4,       5,       6,
                   7,       8,       9,
             ...
             2370575, 2370576, 2370577, 2370578, 2370579, 2370580, 2370581,
             2370582, 2370583, 2370584],
            dtype='int64', length=2370585),
 Index([u'asin', u'salesRank', u'imUrl', u'categories', u'title',
        u'description', u'related', u'price', u'brand'],
       dtype='object')]

In [22]:
metadata.size

21335265

To begin, the team used preprocessing techniques and a vectorizer to decompose the synopses and ratings.

In [2]:
# partition & discard unwanted data (we just want one genre?)

Once the data was imported correctly, the team partitioned the data into a training, development, and test set.

In [ ]:
# preprocessing

Then, the team operationalized the definition of "success" as it pertains to the books and the available data.

In [ ]:
# define a function for success

Finally, the team constructed a classifier to determine if success can be predicted from synopsis

In [ ]:
# classifier

The team found that...

Some alternative methods/explorations...